In [3]:
from matplotlib import pyplot as plt
from scipy.special import softmax
from itertools import product
import numpy as np
from utils import *
from model import *
import time
import seaborn as sns
from glob import glob as glob
sns.set_context('talk')

%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
gsname = 'gs1102'

### full gs data

In [18]:
## run make_df-1102.py to make dfs
datadf = pd.read_csv('data/%s-datadf.csv'%gsname)
gsdf = pd.read_csv('data/%s-summdf.csv'%gsname)

In [19]:
gsdf

,Unnamed: 0,concentration,stickiness_wi,stickiness_bt,sparsity,pvar,lrate,lratep,decay_rate,testacc-b,testacc-e,testacc-i,testacc-l,testacc-m
0,0,0.009917,31.218705,64.725219,0.858954,0,1,1,1,0.501696,0.499693,0.500782,0.500538,0.500527
1,0,0.026625,37.440990,99.023722,0.361747,0,1,1,1,0.499247,0.498278,0.500056,0.499931,0.502273
2,0,0.046686,71.423896,64.692210,0.276881,0,1,1,1,0.499275,0.498438,0.499974,0.499358,0.498034
3,0,0.049736,29.214597,62.200671,1.149741,0,1,1,1,0.498479,0.500777,0.500364,0.499677,0.499613
4,0,0.066967,52.305891,60.336711,0.228389,0,1,1,1,0.498557,0.499550,0.501408,0.502156,0.500616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7445,0,99.946853,88.453906,5.598005,0.998883,0,1,1,1,0.500000,0.500000,0.500000,0.500000,0.500000
7446,0,99.946951,85.812111,44.040495,0.136622,0,1,1,1,0.500000,0.500000,0.500000,0.500000,0.500000
7447,0,99.959699,33.928206,60.207821,0.705893,0,1,1,1,0.500000,0.500000,0.500000,0.500000,0.500000
7448,0,99.966251,99.512375,52.855760,0.366124,0,1,1,1,0.500000,0.500000,0.500000,0.500000,0.500000


In [10]:
datadf.columns
paramL = ['concentration', 'stickiness_wi', 'stickiness_bt',
       'sparsity', 'pvar', 'lrate', 'lratep', 'decay_rate']

### compute df metrics, form summary df

In [11]:
# #### package this. set option to load presaved df
# ## compute summary df
# groupvars = paramL 
# gsdf_group = datadf.groupby(groupvars)
# dfL = []
# for params_i,df_i in gsdf_group:
#   dataD = {**dict(zip(groupvars,params_i))}
#   ## loop conditions (BIEML)
#   for cond_i,df_c in df_i.groupby('cond'):
#     ## compute metrics
#     testacc = np.mean(df_c.acc[-40:])
#     ## populate dataD
#     dataD['testacc-%s'%cond_i[0]] = testacc
#   # 
#   dfL.append(pd.DataFrame(index=[0],data=dataD))
# gsdf = pd.concat(dfL)


In [ ]:
### save gsdf: takes really long to run
gsdf.to_csv('data/%s-summdf.csv'%gsname)
### 

In [ ]:
## subset of dataframe containing relevant params
Btest = gsdf.loc[:,'testacc-b'] > 0.8
Itest1 = gsdf.loc[:,'testacc-i'] > 0.5
Itest2 = gsdf.loc[:,'testacc-i'] < 0.65
Etest = gsdf.loc[:,'testacc-e'] > 0.8
Mtest = gsdf.loc[:,'testacc-m'] > 0.8
Ltest = gsdf.loc[:,'testacc-l'] > 0.8
MItest = gsdf.loc[:,'testacc-m'] - gsdf.loc[:,'testacc-i'] > 0.1
LItest = gsdf.loc[:,'testacc-l'] - gsdf.loc[:,'testacc-i'] > 0.1

C = Btest&Itest2&Itest1&MItest
sgsdf = gsdf[C]
sgsdf

### plt single condition

In [ ]:
## select single param for plotting full acc trace
row = sgsdf.iloc[-1]

query_str = " and ".join(
  ["%s == %f"%(k,v) for k,v in dict(row[:8].round(4)).items()]
)

## df of single parameter condition
cond_df = datadf.round(4).query(query_str)

In [ ]:
f,axar = plt.subplots(1,1,figsize=(20,8))
ax=axar
for c,cdf in cond_df.groupby('cond'):
  ax.plot(np.arange(200),cdf.acc,label=c)
ax.set_ylim(0,1)
plt.legend()